# Working with Pandas dataframs - cleaning data

Register to get your API key and download data [here](https://pypi.org/project/wwo-hist/)

Obtain Historical Weather Forecast data in CSV format using Python, [medium](https://towardsdatascience.com/obtain-historical-weather-forecast-data-in-csv-format-using-python-5a6c090fc828)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import time

## Load data into a Pandas dataframe

In [2]:
fiN = './data/berlin.csv'#-01-MAY-2018-01-SEP-2018.csv'
df0 = pd.read_csv(fiN)
nr, nc = np.shape(df0)
print("n rows {}\nn cols {}".format(nr, nc))
print(df0.columns)
df0.head()

n rows 87672
n cols 25
Index(['date_time', 'maxtempC', 'mintempC', 'totalSnow_cm', 'sunHour',
       'uvIndex', 'uvIndex.1', 'moon_illumination', 'moonrise', 'moonset',
       'sunrise', 'sunset', 'DewPointC', 'FeelsLikeC', 'HeatIndexC',
       'WindChillC', 'WindGustKmph', 'cloudcover', 'humidity', 'precipMM',
       'pressure', 'tempC', 'visibility', 'winddirDegree', 'windspeedKmph'],
      dtype='object')


,date_time,maxtempC,mintempC,totalSnow_cm,sunHour,uvIndex,uvIndex.1,moon_illumination,moonrise,moonset,...,WindChillC,WindGustKmph,cloudcover,humidity,precipMM,pressure,tempC,visibility,winddirDegree,windspeedKmph
0,2009-09-01 00:00:00,32,16,0.0,11.6,6,0,83,06:39 PM,02:28 AM,...,18,29,2,59,0.0,1017,17,10,148,15
1,2009-09-01 01:00:00,32,16,0.0,11.6,6,0,83,06:39 PM,02:28 AM,...,18,25,2,60,0.0,1017,17,10,147,14
2,2009-09-01 02:00:00,32,16,0.0,11.6,6,0,83,06:39 PM,02:28 AM,...,18,22,1,62,0.0,1016,16,10,146,14
3,2009-09-01 03:00:00,32,16,0.0,11.6,6,0,83,06:39 PM,02:28 AM,...,18,19,0,64,0.0,1016,16,10,145,13
4,2009-09-01 04:00:00,32,16,0.0,11.6,6,0,83,06:39 PM,02:28 AM,...,18,20,0,63,0.0,1015,16,10,148,14


### Inspect columns (weather variables)

Let's summarise the informaiton in the columns.

Information about these variables [here](https://www.worldweatheronline.com/developer/api/docs/local-city-town-weather-api.aspx)

In [3]:
time_cols = ['date_time' ]
print("Time ({}): {}, index {}".format(len(time_cols), time_cols, np.argwhere(np.in1d(df0.columns, time_cols))))

temp_cols = ['maxtempC', 'mintempC',  'tempC' ]
print("Temperature ({}): {}".format(len(temp_cols), temp_cols))

thSens_cols = ['FeelsLikeC', 'HeatIndexC']
print("Thermal sensation ({}): {} ".format(len(thSens_cols), thSens_cols))

## Astronomical
sun_cols = ['sunHour', 'uvIndex', 'uvIndex.1', 'sunrise', 'sunset']
print("Sun ({}): {}".format(len(sun_cols), sun_cols))

moon_cols = ['moon_illumination', 'moonrise', 'moonset']
print("Moon ({}): {}".format(len(moon_cols), moon_cols))

wind_cols = ['WindChillC', 'WindGustKmph', 'winddirDegree', 'windspeedKmph']
print("Wind ({}): {}".format(len(wind_cols), wind_cols))

oThermodynamic_cols = set(df0.columns) - \
                      set(time_cols + temp_cols + thSens_cols + sun_cols + moon_cols + wind_cols)
#['totalSnow_cm', 'cloudcover', 'humidity', 'precipMM','pressure', 'visibility']
print("Other - moisture ({}): {}".format(len(oThermodynamic_cols), oThermodynamic_cols))

#print("Athmospheric variables ({}, {}-{}) {}".format(df.columns[:]))

Time (1): ['date_time'], index [[0]]
Temperature (3): ['maxtempC', 'mintempC', 'tempC']
Thermal sensation (2): ['FeelsLikeC', 'HeatIndexC'] 
Sun (5): ['sunHour', 'uvIndex', 'uvIndex.1', 'sunrise', 'sunset']
Moon (3): ['moon_illumination', 'moonrise', 'moonset']
Wind (4): ['WindChillC', 'WindGustKmph', 'winddirDegree', 'windspeedKmph']
Other - moisture (7): {'pressure', 'visibility', 'humidity', 'precipMM', 'DewPointC', 'totalSnow_cm', 'cloudcover'}


We can access the indices of the colums `np.argwhere(np.in1d(df.columns, names_arr))`

### Inspect the rows (time stamps)

Let's have a look at the first and last time stamps in df0.

In [4]:
print("First:\n", df0['date_time'].iloc[0:3], "\nLast:\n", df0['date_time'].iloc[nr-3:])

First:
 0    2009-09-01 00:00:00
1    2009-09-01 01:00:00
2    2009-09-01 02:00:00
Name: date_time, dtype: object 
Last:
 87669    2019-09-01 21:00:00
87670    2019-09-01 22:00:00
87671    2019-09-01 23:00:00
Name: date_time, dtype: object


Between 2009 and 2019, you have 10 years, you are expected to 365 * 10, plus 2 extra days from the leap years. Plus one day more because, we go until the end of the same day we started with.

In [5]:
print("Number of expected days from between 2009-09-01 00:00:00 and 2019-09-01: {}"
      "\nNumber of days calculated from the number of rows in df0: {}".format( 365*10+2+1, (87671+1)/24))

Number of expected days from between 2009-09-01 00:00:00 and 2019-09-01: 3653
Number of days calculated from the number of rows in df0: 3653.0


### Parse `date_time` for year, month and other temporal variables

1. Add daytime columns and initialize them with an empty string

In [6]:
time_cols = ['year', 'month', 'year_day', 'month_day', 'time_hr']

for col in time_cols:
    df0[col] = ''

df0.head(2)

,date_time,maxtempC,mintempC,totalSnow_cm,sunHour,uvIndex,uvIndex.1,moon_illumination,moonrise,moonset,...,pressure,tempC,visibility,winddirDegree,windspeedKmph,year,month,year_day,month_day,time_hr
0,2009-09-01 00:00:00,32,16,0.0,11.6,6,0,83,06:39 PM,02:28 AM,...,1017,17,10,148,15,,,,,
1,2009-09-01 01:00:00,32,16,0.0,11.6,6,0,83,06:39 PM,02:28 AM,...,1017,17,10,147,14,,,,,


### Parse temporal information from `data_time` and add it to the dataframe

You can do this looping over the rows of the dataframe

```
for ix, col in df0[:10].iterrows():
    dtime = col['date_time']
    print(dtime)
    x = time.strptime(dtime, '%Y-%m-%d %H:%M:%S')
    col['year'] = x[0]
    col['month'] = x[1]
    col['month_day'] = x[2]
    col['time_hr'] = x[3]
    df0['year_day'] = x[7]

df0.head()
```

But this is a very unefficient way to do this, because you are using loops on an dataframe. You can improve this process by thinking in the vectorial way (See heisler2018beginner).

[heisler2018beginner] Heisler, Sofia. “A Beginner’s Guide to Optimizing Pandas Code for Speed.” Medium, April 19, 2018. https://engineering.upside.com/a-beginners-guide-to-optimizing-pandas-code-for-speed-c09ef2c6a4d6.


For parsing the time information we use
 the time parsing function `strptime`

`
dtime = df0['date_time'].iloc[3]
print(dtime)
x = time.strptime(dtime, '%Y-%m-%d %H:%M:%S')
`

Parse the daytime column and all information to the dataframe at once. 
The function `time.strptime` in not a vectorial function, meaning it only takes single variables as input and does not know what to do if an array is given. Fortunately, Numpy has the `vectorize` method that transforms simple functions into functions that can take arrays as inputs. The idea of of vectorial functions is very simple, it just consist of having a function that can act over each element of an array and returns as output the outcome of applying the function to each element.

In [7]:
vstrptime = np.vectorize(time.strptime)

In [8]:
df0['year'] = vstrptime(df0['date_time'], '%Y-%m-%d %H:%M:%S')[0]
df0['month'] = vstrptime(df0['date_time'], '%Y-%m-%d %H:%M:%S')[1]
df0['month_day'] = vstrptime(df0['date_time'], '%Y-%m-%d %H:%M:%S')[2]
df0['time_hr'] = vstrptime(df0['date_time'], '%Y-%m-%d %H:%M:%S')[3]
df0['year_day'] = vstrptime(df0['date_time'], '%Y-%m-%d %H:%M:%S')[7]

df0.head()

,date_time,maxtempC,mintempC,totalSnow_cm,sunHour,uvIndex,uvIndex.1,moon_illumination,moonrise,moonset,...,pressure,tempC,visibility,winddirDegree,windspeedKmph,year,month,year_day,month_day,time_hr
0,2009-09-01 00:00:00,32,16,0.0,11.6,6,0,83,06:39 PM,02:28 AM,...,1017,17,10,148,15,2009,9,244,1,0
1,2009-09-01 01:00:00,32,16,0.0,11.6,6,0,83,06:39 PM,02:28 AM,...,1017,17,10,147,14,2009,9,244,1,1
2,2009-09-01 02:00:00,32,16,0.0,11.6,6,0,83,06:39 PM,02:28 AM,...,1016,16,10,146,14,2009,9,244,1,2
3,2009-09-01 03:00:00,32,16,0.0,11.6,6,0,83,06:39 PM,02:28 AM,...,1016,16,10,145,13,2009,9,244,1,3
4,2009-09-01 04:00:00,32,16,0.0,11.6,6,0,83,06:39 PM,02:28 AM,...,1015,16,10,148,14,2009,9,244,1,4


### Subset to have a daily frequency

We have data with an hourly frequency, let's resample to the daily rate and keep only the max and min temp and the temporal variables. 
Because the max and the min temperatures are constant for 24hrs, we take a row every 24 hrs.

In [9]:
#A = df0.values
cols = ['date_time', 'maxtempC', 'mintempC', 'year_day', 'year', 'month']
df_dy = df0[cols].iloc[::24]
df_dy

,date_time,maxtempC,mintempC,year_day,year,month
0,2009-09-01 00:00:00,32,16,244,2009,9
24,2009-09-02 00:00:00,24,17,245,2009,9
48,2009-09-03 00:00:00,22,17,246,2009,9
72,2009-09-04 00:00:00,21,14,247,2009,9
96,2009-09-05 00:00:00,18,11,248,2009,9
...,...,...,...,...,...,...
87552,2019-08-28 00:00:00,33,23,240,2019,8
87576,2019-08-29 00:00:00,31,22,241,2019,8
87600,2019-08-30 00:00:00,28,21,242,2019,8
87624,2019-08-31 00:00:00,32,22,243,2019,8


## Save data

In [11]:
cols = ['mintempC', 'maxtempC', 'year_day', 'year', 'month']

outf = './data/Dailytemp_Berlin_2009-09-01-2019-09-01.csv'
df_dy[cols].to_csv(outf, index=False, header=False)
print(outf)

./data/Dailytemp_Berlin_2009-09-01-2019-09-01.csv
